In [1]:
import numpy as np
import xarray as xr
from seapopym_optimization.cost_function import NoTransportCostFunction

In [2]:
from seapopym.model.no_transport_model import NoTransportModel

from seapopym.configuration.no_transport.configuration import NoTransportConfiguration

from seapopym.configuration.no_transport.parameter import (
    ForcingParameters,
    FunctionalGroups,
    KernelParameters,
    NoTransportParameters,
)

from seapopym.configuration.parameters.parameter_forcing import ForcingUnit

from seapopym.configuration.parameters.parameter_functional_group import (
    FunctionalGroupUnit,
    FunctionalGroupUnitMigratoryParameters,
    FunctionalGroupUnitRelationParameters,
)

from seapopym.standard.units import StandardUnitsLabels

Load forcing.


In [3]:
time_start, time_end = "1998-01-01", "2022-01-01"

hot_data = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cmems.zarr", engine="zarr")
hot_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
hot_data = hot_data.sel(time=slice(time_start, time_end))
_ = hot_data.load()

cafe_npp = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.lat.attrs = {"units": "degrees_north", "long_name": "latitude", "axis": "Y"}
cafe_npp.lon.attrs = {"units": "degrees_east", "long_name": "longitude", "axis": "X"}
cafe_npp = cafe_npp.rename({"lat": "latitude", "lon": "longitude"})
_ = cafe_npp.load()

hot_obs = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_obs.zarr", engine="zarr")
hot_obs = hot_obs.sel(time=slice(time_start, time_end))
_ = hot_obs.load()

Create structure for SeapoPym simulation.


In [4]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=hot_data["T"], resolution=1/12),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1/12),
)

2024-09-25 15:13:46,072 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Cost function


In [5]:
fixed_parameters = [
    [None, None, None, None, 1, 1, None],
]
cost_function = NoTransportCostFunction(
    nb_parameters=7,
    nb_functional_groups=1,
    fixed_parameters=fixed_parameters,
    forcing_parameters=forcing_parameters,
    observations=hot_obs,
    groups_name=["D1N1"]
).generate()

In [7]:
cost = cost_function([11, 0.1, 100, -0.1, 0.2])

2024-09-25 15:14:19,483 :: Seapodym ::  WARNING ::
|	Parameter inv_lambda_rate : -0.1 has a negative value. It means that the mortality is decreasing when temperature increase. Do you mean to use a positive value?

2024-09-25 15:14:19,490 :: Seapodym ::  WARNING ::
|	Parameter temperature_recruitment_rate : 0.1 has a positive value. It means that the recruitment time is increasing with temperature. Do you mean to use a negative value?

2024-09-25 15:14:19,536 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-09-25 15:14:19,545 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-09-25 15:14:19,552 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-09-25 15:14:19,587 :: Seapodym ::  DEBUG ::
|	Direct computation for _average_temperature.

2024-09-25 15:14:19,602 :: Seapodym ::  DEBUG ::
|	Direct computation for _apply_coefficient_to_primary_production_helper.

2024-09-25 15:14:19,607 :: Seapodym ::  DEBUG ::


In [8]:
cost

(<xarray.DataArray <ForcingLabels.biomass: 'biomass'> (functional_group: 1,
                                                       time: 8767, latitude: 1,
                                                       longitude: 1)> Size: 70kB
 array([[[[5.24929626e-05]],
 
         [[1.03853422e-04]],
 
         [[1.54081029e-04]],
 
         ...,
 
         [[4.30035359e-02]],
 
         [[4.30006742e-02]],
 
         [[4.29981499e-02]]]])
 Coordinates:
   * functional_group  (functional_group) int64 8B 0
   * latitude          (latitude) float64 8B 31.6
   * longitude         (longitude) float64 8B -64.2
   * time              (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
 Attributes:
     long_name:    biomass
     units:        kilogram / meter ** 2
     description:  The biomass of the recruited individuals.,)